In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from glob import glob
from ALLCools.mcds import RegionDS


In [6]:
group_name = 'AMY'


In [2]:
motif = xr.open_zarr('/cemba/wmb/genome/CEMBA.DMRGroups.Motif.zarr/motif')
dmr_id = motif.get_index('dmr_id')
dmr_map = pd.read_hdf('/home/jzhou_salk_edu/sky_workdir/230209_dmrmotif/dmr_idmap.hdf', key='data')


In [7]:
region = group_name
region_ds = RegionDS.open(sorted(glob(f'/rs2/221126-dmr/*/*/{region}/{region}-chr*.dmr.ds')), region_dim='dmr')
seldmr = region_ds.get_index('dmr').map(dmr_map)
seldmr = dmr_id.intersection(seldmr)
seldmr.name = 'dmr_id'
motif_reg = motif.sel({'dmr_id':seldmr})


In [8]:
chunk_size = 24
motif_id = motif_reg.get_index('motif')
rng = np.random.default_rng(seed=0)
random_permutations = rng.permutation(seldmr.size)


In [ ]:
for chunk_start in range(0, motif_id.size, chunk_size):
    motif_tmp = motif_id[chunk_start:chunk_start+chunk_size]
    all_score = motif_reg.sel({'motif':motif_tmp})['score'].load()
    all_rank = random_permutations[(-all_score)[:, random_permutations].argsort(axis=1).values].argsort(axis=1)
    all_rank = pd.DataFrame(all_rank, index=motif_tmp, columns=seldmr)
    all_rank = all_rank.astype("uint32")

    ds = xr.Dataset({# "score": all_score, 
                     "rank": all_rank})
    # ds['score'].encoding = {'chunks': (chunk_size, 1000000)}
    ds['rank'].encoding = {'chunks': (chunk_size, 1000000)}
    ds.coords['motif'].encoding = {'chunks': (5000,)}
    ds.coords['dmr_id'].encoding = {'chunks': (10000000,)}
    if chunk_start == 0:
        ds.to_zarr(f'{region}_motif.ds', mode='w')
    else:
        ds.to_zarr(f'{region}_motif.ds', append_dim='motif')
    print(chunk_start)


0
24
48
72
